In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os

In [2]:
source_folder = './data'
destination_folder = './model'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field),  ('text', text_field)]

In [5]:
train, valid = TabularDataset.splits(path=source_folder, train='train_char_based.csv', validation='valid_char_based.csv',
                                     format='CSV', fields=fields, skip_header=True)
print('Object:', train[0])
print('key:', train[0].__dict__.keys())
print('text:', train[0].text)
print('label:', train[0].label)
for i in train:
    print(i.label)
train_iter = BucketIterator(train, batch_size=8, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=8, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)

Object: <torchtext.data.example.Example object at 0x000001D7B79DDF48>
key: dict_keys(['label', 'text'])
text: [101, 1048, 6182, 6199, 14839, 1056, 1044, 1041, 1054, 1041, 2686, 1037, 1054, 1041, 2686, 1052, 1041, 1052, 1048, 1041, 2686, 1055, 29215, 12162, 13153, 14839, 2067, 2067, 2067, 2067, 2067, 1051, 1052, 1048, 1041, 2686, 1052, 2067, 1055, 1037, 1045, 1048, 1045, 1050, 1043, 2686, 1038, 1051, 1037, 1056, 1055, 2686, 1045, 1050, 1056, 1044, 1041, 2686, 102]
label: 13
13
62
53
128
97
0
10
111
12
50
72
121
28
115
133
35
136
103
32
13
33
42
57
36
65
58
124
143
54
51
57
31
117
115
54
130
90
1
45
34
46
104
69
127
134
24
90
68
62
93
74
68
98
41
43
41
138
98
6
96
13
145
8
98
134
109
71
21
63
137
18
99
143
7
4
43
105
70
103
17
75
103
38
113
98
92
66
40
30
91
106
68
123
87
40
74
55
116
68
69
76
144
83
93
92
7
144
51
8
19
37
103
129
37
118
73
94
9
68
62
10
79
120
99
24
40
122
73
100
146
42
143
134
90
27
9
84
13
83
4
120
79
115
27
15
35
133
60
68
16
133
37
134
142
117
85
124
65
126
9
54
81


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



19
56
11
36
94
0
125
9
57
137
97
20
84
117
21
129
91
110
74
21
11
7
0
140
101
94
66
146
16
2
85
23
117
104
100
79
79
58
139
79
88
53
2
134
68
62
77
68
78
123
15
107
97
21
143
20
0
24
68
28
72
35
60
60
87
129
9
2
146
88
29
103
107
122
140
1
88
92
117
54
29
69
61
91
80
92
114
14
62
66
125
143
41
92
53
128
106
7
33
118
100
44
22
5
145
13
106
56
110
85
105
92
9
64
31
93
69
52
85
147
27
95
0
57
0
88
120
80
1
36
9
25
95
72
104
34
93
55
88
39
67
12
90
24
108
54
126
47
66
132
136
0
73
92
22
3
107
18
37
10
96
120
35
97
42
47
96
134
138
14
77
137
74
72
127
19
135
14
84
105
40
146
111
136
60
142
6
67
106
69
103
47
14
20
65
132
104
69
29
10
76
32
65
64
2
118
11
73
133
96
131
66
53
19
60
77
15
23
79
108
44
19
3
132
85
105
97
83
25
19
146
49
43
104
126
103
66
5
89
37
35
55
69
124
56
69
55
126
117
113
51
39
67
9
119
144
126
88
20
82
144
3
137
108
26
106
55
46
53
17
40
84
137
63
49
91
114
58
60
6
61
44
27
51
0
35
49
55
20
82
112
72
39
129
0
93
90
48
133
2
3
44
122
45
50
51
109
102
79
119
39
48
115
101

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




146
107
52
138
34
51
31
34
92
36
5
123
83
72
7
76
28
60
106
43
113
81
50
2
58
0
117
27
92
45
40
147
143
50
139
16
5
147
53
4
13
93
51
60
136
74
142
60
34
21
147
32
146
30
121
96
136
107
129
37
134
132
117
83
49
118
144
145
50
5
8
138
13
6
36
129
145
42
69
106
129
13
89
146
111
145
103
92
27
62
45
98
80
21
54
47
20
122
104
83
96
29
61
49
18
145
21
51
59
49
124
124
119
60
43
55
105
146
127
114
71
54
18
132
12
113
93
22
22
3
54
93
61
139
144
95
75
58
14
92
58
27
3
86
115
51
3
43
39
16
129
125
79
55
137
84
64
89
49
102
53
128
67
103
80
62
95
46
98
76
9
44
36
61
10
10
28
76
13
6
137
145
50
68
89
59
82
11
1
17
37
110
101
69
139
133
131
126
80
9
29
2
145
15
145
89
58
25
85
99
101
73
22
10
65
22
101
50
54
30
82
40
31
127
142
15
11
61
136
133
25
146
25
6
27
85
98
58
17
132
21
104
96
62
116
108
121
128
127
93
13
105
0
94
20
20
93
41
48
107
38
49
43
128
10
61
145
61
11
8
82
96
5
85
30
123
124
133
106
27
1
92
129
86
19
81
33
61
64
52
14
1
93
20
142
40
92
79
7
11
103
47
61
140
55
12
27
132
101
44


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




110
99
114
100
27
55
98
116
84
103
61
53
123
37
21
8
136
61
136
2
106
45
28
102
128
110
2
138
68
70
55
143
118
140
90
21
11
4
0
72
135
77
45
128
108
69
50
53
134
105
33
10
61
145
106
146
92
3
99
116
49
58
42
85
1
49
81
127
146
7
99
114
56
34
48
82
13
144
101
119
78
69
73
63
49
77
25
19
38
84
76
129
130
110
42
20
78
103
116
144
111
122
98
117
146
79
38
128
8
4
67
7
70
67
0
140
83
22
118
96
65
33
61
35
38
84
147
43
14
91
111
126
7
76
25
45
84
140
116
63
115
69
12
139
58
13
14
100
101
43
140
77
43
91
135
90
90
118
14
10
51
58
36
99
13
30
108
21
41
140
63
32
146
110
73
36
50
16
69
50
117
62
125
39
133
35
64
20
92
62
7
12
5
79
90
27
2
113
50
40
60
143
86
36
94
31
63
100
91
112
85
111
110
0
82
3
79
60
33
86
25
87
46
100
94
92
56
31
20
147
8
72
30
80
20
90
88
88
110
123
126
72
119
96
56
144
24
67
130
118
84
87
12
97
78
115
72
59
99
139
69
143
50
129
147
11
2
19
94
129
85
55
51
10
100
36
110
118
13
97
115
89
8
2
59
39
136
59
31
88
11
104
12
132
18
60
137
80
45
136
19
55
76
59
104
30
39
115
13

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



74
2
55
31
71
77
70
88
69
46
129
14
24
64
73
80
24
72
96
143
51
116
51
83
142
117
145
92
60
79
112
68
136
80
68
139
117
126
100
52
102
142
109
53
92
117
95
89
72
48
132
129
22
104
93
122
49
13
92
84
61
10
146
132
111
88
96
84
134
116
73
79
54
30
25
92
47
65
61
20
100
135
103
88
60
36
123
39
69
81
115
101
76
89
57
86
138
140
8
50
136
56
14
65
82
101
99
97
39
127
81
3
127
145
70
86
57
145
3
137
62
49
94
93
50
34
101
19
127
87
73
50
10
67
17
51
63
113
147
12
113
109
76
65
31
101
99
10
80
73
44
0
106
18
74
98
118
136
83
145
110
25
130
80
129
51
97
46
61
116
67
89
57
91
17
115
52
10
135
129
50
88
106
31
58
27
110
47
98
106
118
8
65
40
83
5
147
61
40
114
67
146
104
76
115
73
90
42
45
119
144
20
117
130
103
109
1
24
62
84
135
60
99
117
66
4
122
104
86
136
115
95
12
8
7
68
100
132
13
89
45
41
54
85
4
61
106
147
107
136
85
127
144
109
73
111
90
6
14
56
3
64
119
68
10
60
78
51
51
86
15
129
69
147
68
115
100
79
6
43
84
58
124
134
93
145
99
53
26
144
6
145
33
3
81
72
64
79
1
12
33
120
33
125
7
147

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()
        # https://huggingface.co/models
        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name, num_labels=148)

    def forward(self, text, label):
        # self.encoder.num_labels = 148
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [7]:
# Save and Load Functions
def save_checkpoint(save_path, model, valid_loss):
    if save_path == None:
        return

    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model):
    if load_path == None:
        return

    state_dict = torch.load(load_path, map_location=device)  # put model param to device (gpu or cpu)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):
    if save_path == None:
        return

    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    
    
def load_metrics(load_path):
    if load_path == None:
        return

    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [8]:
def train(model,
          optimizer,
          criterion=nn.BCELoss(),
          train_loader=train_iter,
          valid_loader=valid_iter,
          num_epochs=10,
          eval_every=len(train_iter) // 2,
          file_path=destination_folder,
          best_train_loss=float("Inf")):
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for (labels, text), _ in train_loader:
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            text = text.type(torch.LongTensor)
            text = text.to(device)
            output = model(text, labels)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():
                    # validation loop
                    for (labels, text), _ in valid_loader:
                        labels = labels.type(torch.LongTensor)
                        labels = labels.to(device)
                        text = text.type(torch.LongTensor)
                        text = text.to(device)
                        output = model(text, labels)
                        loss, _ = output

                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch + 1, num_epochs, global_step, num_epochs * len(train_loader),
                              average_train_loss, average_valid_loss))

                # checkpoint
                if best_train_loss > average_train_loss:
                    best_train_loss = average_train_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_train_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)

    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

In [9]:
model = BERT().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

train(model=model, optimizer=optimizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch [1/10], Step [930/18610], Train Loss: 4.9467, Valid Loss: 5.0155
Model saved to ==> ./model/model.pt
Model saved to ==> ./model/metrics.pt
Epoch [1/10], Step [1860/18610], Train Loss: 4.9705, Valid Loss: 5.0033
Epoch [2/10], Step [2790/18610], Train Loss: 4.8990, Valid Loss: 5.0279
Model saved to ==> ./model/model.pt
Model saved to ==> ./model/metrics.pt
Epoch [2/10], Step [3720/18610], Train Loss: 4.9755, Valid Loss: 5.0428
Epoch [3/10], Step [4650/18610], Train Loss: 4.8869, Valid Loss: 5.0396
Model saved to ==> ./model/model.pt
Model saved to ==> ./model/metrics.pt
Epoch [3/10], Step [5580/18610], Train Loss: 4.9608, Valid Loss: 5.0636
Epoch [4/10], Step [6510/18610], Train Loss: 4.8704, Valid Loss: 5.0564
Model saved to ==> ./model/model.pt
Model saved to ==> ./model/metrics.pt
Epoch [4/10], Step [7440/18610], Train Loss: 4.9545, Valid Loss: 5.0775
Epoch [5/10], Step [8370/18610], Train Loss: 4.8757, Valid Loss: 5.0329
Epoch [5/10], Step [9300/18610], Train Loss: 4.9552, Vali

KeyboardInterrupt: 

In [10]:
# Evaluation Function

def evaluate(model, test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for (labels, text), _ in test_loader:

                labels = labels.type(torch.LongTensor)           
                labels = labels.to(device)
                text = text.type(torch.LongTensor)  
                text = text.to(device)
                output = model(text, labels)

                _, output = output
                y_pred.extend(torch.argmax(output, 1).tolist())
                y_true.extend(labels.tolist())
    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=[i for i in range(148)], digits=4))

In [11]:
best_model = BERT().to(device)

load_checkpoint(destination_folder + '/model.pt', best_model)

evaluate(best_model, train_iter)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Model loaded from <== ./model/model.pt
Classification Report:


C:\Users\pocheng.lin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       157
           1     0.0000    0.0000    0.0000        94
           2     0.0000    0.0000    0.0000       126
           3     0.0000    0.0000    0.0000       103
           4     0.0000    0.0000    0.0000        96
           5     0.0000    0.0000    0.0000        95
           6     0.0000    0.0000    0.0000        93
           7     0.0000    0.0000    0.0000       100
           8     0.0091    1.0000    0.0181       136
           9     0.0000    0.0000    0.0000       122
          10     0.0000    0.0000    0.0000       116
          11     0.0000    0.0000    0.0000        74
          12     0.0000    0.0000    0.0000       121
          13     0.0000    0.0000    0.0000       146
          14     0.0000    0.0000    0.0000       125
          15     0.0000    0.0000    0.0000       107
          16     0.0000    0.0000    0.0000       106
          17     0.0000    